# 📊 Лабораторная работа №2: DFT/FFT и спектральный анализ
**Курс:** Основы теории сигналов и систем (ОТСиС)  
**Цель:** Освоить преобразование Фурье, спектральный анализ и эффекты дискретизации.

## 🎯 Ключевые вопросы
1. Что такое DFT (дискретное преобразование Фурье)?
2. Как работает FFT (быстрое преобразование Фурье)?
3. Как интерпретировать спектр?
4. Что такое утечка спектра и как её избежать?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq, ifft

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
print('✅ Библиотеки загружены!')

## 1. Дискретное преобразование Фурье (DFT)

DFT переводит сигнал из временной области в частотную:

$$X[k] = \sum_{n=0}^{N-1} x[n] e^{-j2\pi kn/N}$$

Где:
- $x[n]$ — исходный сигнал (N отсчетов)
- $X[k]$ — спектр (N комплексных значений)
- $k$ — номер частотного бина (0 до N-1)

**Важно:** Спектр содержит информацию об амплитуде И фазе каждой частотной компоненты.

### Амплитудный спектр:
$$|X[k]| = \sqrt{\text{Real}^2 + \text{Imag}^2}$$

### Фазовый спектр:
$$\angle X[k] = \arctan\left(\frac{\text{Imag}}{\text{Real}}\right)$$

In [ ]:
# Параметры
fs = 1000  # Частота дискретизации
t = np.arange(0, 1.0, 1/fs)  # 1 секунда

# Сигнал: микс из 3 частот
f1, f2, f3 = 10, 25, 50  # Герцы
s = np.sin(2*np.pi*f1*t) + 0.5*np.sin(2*np.pi*f2*t) + 0.3*np.sin(2*np.pi*f3*t)

# DFT (FFT в Python)
N = len(s)
S = fft(s)  # Комплексный спектр
freqs = fftfreq(N, 1/fs)  # Оси частот

# Нормировка
S_normalized = np.abs(S) / N

# Берем только положительные частоты
freqs_positive = freqs[:N//2]
S_positive = S_normalized[:N//2]

# Визуализация
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# График сигнала
ax1.plot(t[:100], s[:100], 'b', linewidth=1.5)
ax1.set_title('Исходный сигнал (первые 100 отсчетов)')
ax1.set_xlabel('Время, с')
ax1.set_ylabel('Амплитуда')
ax1.grid(True)

# Спектр
ax2.plot(freqs_positive, S_positive, 'r', linewidth=1.5)
ax2.set_title('Амплитудный спектр (FFT)')
ax2.set_xlabel('Частота, Гц')
ax2.set_ylabel('Амплитуда (нормированная)')
ax2.grid(True)
ax2.set_xlim([0, 100])

plt.tight_layout()
plt.show()

print(f'Пики спектра должны быть на частотах: {f1}, {f2}, {f3} Гц')

## 2. Утечка спектра (Spectral Leakage)

**Проблема:** Если длина сигнала не кратна периоду, энергия "размазывается" по спектру.

**Решение:** Используем **оконные функции** (windows), которые плавно обнуляют края сигнала.

Популярные окна:
- **Hamming:** $w[n] = 0.54 - 0.46\cos(2\pi n/N)$
- **Hann:** $w[n] = 0.5 - 0.5\cos(2\pi n/N)$
- **Blackman:** более сложная, но лучше подавляет боковые лепестки

In [ ]:
# Сигнал, который не кратен периоду (вызывает утечку)
fs = 1000
duration = 0.95  # НЕ целая секунда!
t = np.arange(0, duration, 1/fs)
f = 10  # Частота 10 Гц (идеальная)
s_noperiodic = np.sin(2*np.pi*f*t)

# FFT без окна и с окнами
N = len(s_noperiodic)
freqs = fftfreq(N, 1/fs)[:N//2]

# Без окна
S_no_window = np.abs(fft(s_noperiodic))[:N//2] / N

# С окном Hamming
window_hamming = signal.hamming(N)
s_windowed = s_noperiodic * window_hamming
S_hamming = np.abs(fft(s_windowed))[:N//2] / N

# Визуализация
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Исходный сигнал
axes[0, 0].plot(t, s_noperiodic, 'b')
axes[0, 0].set_title('Исходный сигнал (не кратен периоду)')
axes[0, 0].set_xlabel('Время, с')
axes[0, 0].grid(True)

# Оконный сигнал
axes[0, 1].plot(t, s_windowed, 'g')
axes[0, 1].set_title('После применения окна Hamming')
axes[0, 1].set_xlabel('Время, с')
axes[0, 1].grid(True)

# Спектр без окна (утечка!)
axes[1, 0].semilogy(freqs, S_no_window)
axes[1, 0].set_title('Спектр БЕЗ окна (видна утечка)')
axes[1, 0].set_xlabel('Частота, Гц')
axes[1, 0].set_xlim([0, 50])
axes[1, 0].grid(True)

# Спектр с окном (чище)
axes[1, 1].semilogy(freqs, S_hamming)
axes[1, 1].set_title('Спектр С окном Hamming (утечка подавлена)')
axes[1, 1].set_xlabel('Частота, Гц')
axes[1, 1].set_xlim([0, 50])
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 🛠 ЗАДАНИЕ: Спектральный анализ реального звука

**Задача:** Загрузить аудиофайл (или сгенерировать) и проанализировать его спектр.

**Требования:**
1. Создать сигнал или загрузить audio (см. ниже)
2. Применить окно Hann
3. Вычислить FFT
4. Построить амплитудный спектр (логарифмическая шкала)
5. Найти 5 главных пиков и их частоты

In [ ]:
# Сгенерируем "голос" как микс частот
fs = 8000
t = np.arange(0, 2.0, 1/fs)

# Синтетический "голос": основная частота 200 Гц + гармоники
f0 = 200  # Основная частота
harmonics = f0 * np.array([1, 2, 3, 4, 5])  # Гармоники
amplitudes = np.array([1.0, 0.5, 0.3, 0.2, 0.1])

s_voice = np.zeros_like(t)
for h, amp in zip(harmonics, amplitudes):
    s_voice += amp * np.sin(2*np.pi*h*t)

# Добавим шум
s_voice += 0.05 * np.random.randn(len(t))

# Применяем окно Hann
window = signal.hann(len(s_voice))
s_windowed = s_voice * window

# FFT
N = len(s_windowed)
S = np.abs(fft(s_windowed)) / N
freqs = fftfreq(N, 1/fs)[:N//2]
S = S[:N//2]

# Находим топ-5 пиков
top_indices = np.argsort(S)[-5:][::-1]
top_freqs = freqs[top_indices]
top_amps = S[top_indices]

# Визуализация
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Волна
ax1.plot(t[:1000], s_voice[:1000], 'b', linewidth=0.5, label='Сигнал')
ax1.plot(t[:1000], window[:1000], 'r--', linewidth=1, label='Окно Hann')
ax1.set_title('Синтетический "голос" с окном')
ax1.set_xlabel('Время, с')
ax1.set_ylabel('Амплитуда')
ax1.legend()
ax1.grid(True)

# Спектр (логарифмическая шкала)
ax2.semilogy(freqs, S, 'g', linewidth=1)
ax2.scatter(top_freqs, top_amps, color='r', s=100, zorder=5, label='Топ-5 пиков')
ax2.set_title('Спектр "голоса" (логарифмическая шкала)')
ax2.set_xlabel('Частота, Гц')
ax2.set_ylabel('Амплитуда')
ax2.set_xlim([0, 1000])
ax2.grid(True, which='both', alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

# Вывод результатов
print("\n=== ТОП-5 ПИКОВ СПЕКТРА ===")
for i, (f, amp) in enumerate(zip(top_freqs, top_amps), 1):
    print(f"{i}. Частота: {f:.1f} Гц, Амплитуда: {amp:.4f}")